In [1]:
import pandas as pd
import vectorbt as vbt
import numpy as np

In [2]:
# test with data
ticker = "TSLA"
start_date = "2019-01-01"
end_date = "2024-01-01"

price_1d = vbt.YFData.download(ticker,
        missing_index='drop',
        start=start_date,
        end=end_date,
        interval="1D").get('Close')

price_1d

Date
2019-01-02 05:00:00+00:00     20.674667
2019-01-03 05:00:00+00:00     20.024000
2019-01-04 05:00:00+00:00     21.179333
2019-01-07 05:00:00+00:00     22.330667
2019-01-08 05:00:00+00:00     22.356667
                                ...    
2023-12-22 05:00:00+00:00    252.539993
2023-12-26 05:00:00+00:00    256.609985
2023-12-27 05:00:00+00:00    261.440002
2023-12-28 05:00:00+00:00    253.179993
2023-12-29 05:00:00+00:00    248.479996
Name: Close, Length: 1258, dtype: float64

In [3]:
# combining indicators

def combine_indicators(close, window = 14, entry = 30, exit = 70, slow = 50, fast = 10):
     trend = np.full(close.shape, np.nan)
     rsi = vbt.RSI.run(close, window = window, short_name="rsi")
     slow_ma = vbt.MA.run(close, slow)
     fast_ma = vbt.MA.run(close, fast)
     ma_signal = fast_ma.ma_above(slow_ma).to_numpy()
     rsi_above = rsi.rsi_above(exit).to_numpy()
     rsi_below = rsi.rsi_below(entry).to_numpy()
     for x in range(len(close)):
         if rsi_above[x] == True and ma_signal[x] == True:
             trend[x] = 1
         elif rsi_below[x] == True or ma_signal[x] == False:
             trend[x] = -1
         else:
             trend[x] = 0
     return trend

# returns 1 for buy signal, -1 for sell signal, and 0 for no signal

In [4]:
myInd = vbt.IndicatorFactory(
         class_name="Combination",
         short_name="comb",
         input_names=["close"],
         param_names=["window","entry","exit","slow","fast"],
         output_names=["ind"]
         ).from_apply_func(
             combine_indicators,
             window=14,
             entry=30,
             exit=70,
             slow=50,
             fast=10
         )


In [5]:
res = myInd.run(price_1d,
                window=14,
                entry=30,
                exit=70,
                slow=50,
                fast=10,
                param_product=True
                )

# running our indicator
entries = res.ind == 1.0
exits = res.ind == -1.0
pf = vbt.Portfolio.from_signals(price_1d, entries, exits, freq = "1D", fees=0.01)

In [6]:
print(pf.stats())

Start                         2019-01-02 05:00:00+00:00
End                           2023-12-29 05:00:00+00:00
Period                               1258 days 00:00:00
Start Value                                       100.0
End Value                                    643.669229
Total Return [%]                             543.669229
Benchmark Return [%]                        1101.857285
Max Gross Exposure [%]                            100.0
Total Fees Paid                              160.277203
Max Drawdown [%]                              54.608428
Max Drawdown Duration                 540 days 00:00:00
Total Trades                                         13
Total Closed Trades                                  13
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  38.461538
Best Trade [%]                               171.470188
Worst Trade [%]                              -16

In [7]:
pf.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'dbc56c0e-0eaa-4003-a121-907cd58ac8e4',
              'x': array([datetime.datetime(2019, 1, 2, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2019, 1, 3, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2019, 1, 4, 5, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2023, 12, 27, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 12, 28, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 12, 29, 5, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([ 20.67466736,  20.02400017,  21.17933273, ..., 261.44000244,
                       

In [8]:
pf.plot(subplots=['cash', 'assets', 'value']).show()


In [ ]:
# use the ouput to create highlevel summary of 
# 1) what the strategy is doing
# 2) how it performed 
# 3) how it compares to benchmark


# let's use MACD as indicator

In [ ]:
# combining indicators

def combine_indicators(close, window = 14, entry = 30, exit = 70, slow = 50, fast = 10):
     trend = np.full(close.shape, np.nan)
     rsi = vbt.RSI.run(close, window = window, short_name="rsi")
     slow_ma = vbt.MA.run(close, slow)
     fast_ma = vbt.MA.run(close, fast)
     ma_signal = fast_ma.ma_above(slow_ma).to_numpy()
     rsi_above = rsi.rsi_above(exit).to_numpy()
     rsi_below = rsi.rsi_below(entry).to_numpy()
     for x in range(len(close)):
         if rsi_above[x] == True and ma_signal[x] == True:
             trend[x] = 1
         elif rsi_below[x] == True or ma_signal[x] == False:
             trend[x] = -1
         else:
             trend[x] = 0
     return trend

# returns 1 for buy signal, -1 for sell signal, and 0 for no signal